# Recalibrating the ringdown attachment frequency for ENIGMA

 - The idea here is to use an MCMC algorithm to find the best parameters that make ENIGMA agree with SEOBNRv4 in the regions of parameter space that overlap for them
 - We will use the emcee package for this
 - We need a few components for MCMC:
    - A way to generate ENIGMA for a given set of parameters **$\theta$**


**Define**:

 1. Fix $\{m_1, m_2\}$
 1. $\theta = \{\omega_\mathrm{attach}, \,\mathrm{PN\, order}\}$
 1. ndim = len(\theta) = 2
 1. log_prob $\propto \langle h_1, h_2 \rangle$ 

**Choose internal parameters**:

 1. PSD $\in$ [**aLIGOZeroDetHighPower**, **Flat**]
 2. low frequency cutoff $\in [15, 20, 30]Hz$
 2. sample rate = $2048$
 2. masses $(m_1, m_2) \in [40, 60]M_\odot$
 2. attachment time $t_\mathrm{attach} \in $ **???**
 2. attachment time window $dt_\mathrm{attach} \in $ **???**

**Algorithm**:

* Sample $\theta$:
 1. PNO $\in [6, 7, 8, 9, 10, 11, 12]$
 1. $\omega_\mathrm{attach} \in [0.01, 0.1]$
* Do:
 1. Compute $h_1 \equiv$ h_ENIGMA$\left(m_1, m_2, \theta; t\right)$
 1. Compute $h_2 \equiv$ h_EOB$\left(m_1, m_2, \theta; t\right)$
 1. Compute log_prob = $\propto \langle h_1, h_2 \rangle$

**Code plan**:

* `gwnrtools_enigma_plan_grid_and_make_dag`
 - Inputs:
   - mass1 range, num_points
   - mass2 range, num_points
   - f_lower choices
   - psd choices
   - num_unique_mcmc_per_job
 - Meat:
   - Generate a hypercubic lattice over {mass1, mass2, f_lower, psd_choices}
   - 
 - Output:
   - DAG
* `gwnrtools_enigma_sample_parameters`
 - Inputs:
   - Fixed values of mass1
   - Fixed values of mass2
   - Fixed values of f_lower
   - Fixed values of psd
 - Meat:
   - Iterate over {mass1, mass2, f_lower, psd_choices}
   - MCMC over $\{\omega_\mathrm{attach}, \mathrm{PNO}\}$
 - Output:
   - Text file with columns:
     - [1] mass-ratio
     - [2] total-mass
     - [3] mass1
     - [4] mass2
     - [5] ecc value
     - [6] PN order
     - [7] omega_attach
     - [8] overlap
     - [9] Re[unphase-maximized overlap]
     - [10] Im[unphase-maximized overlap]
     - [11] norm1
     - [12] norm2

In [1]:
import os
import logging
import numpy as np
import h5py
import emcee

In [2]:
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.psd import from_string
from pycbc.filter import match

In [3]:
import sys
sys.path.append(os.path.join(os.environ['HOME'], 'src/GWNRTools'))

In [4]:
import GWNRTools as gwnrtools
from GWNRTools.Utils import make_padded_frequency_series

In [5]:
logging.getLogger().setLevel(logging.INFO)

In [6]:
run_directory = '/home/prayush/TESTME'

## `gwnrtools_enigma_plan_grid_and_make_dag`

**Code plan**:

* `gwnrtools_enigma_plan_grid_and_make_dag`
 - Inputs:
   - mass1 range, num_points
   - mass2 range, num_points
   - f_lower choices
   - psd choices
   - num_unique_mcmc_per_job
 - Meat:
   - Generate a hypercubic lattice over {mass1, mass2, f_lower, psd_choices}
   - 
 - Output:
   - DAG

In [7]:
!gwnrtools_enigma_plan_calib_grid_and_make_dag --help

usage: /home/prayush/src/GWNRTools/bin//gwnrtools_enigma_plan_calib_grid_and_make_dag [--options]

Setup workflow to perform calibration of ENIGMA model using another BBH model

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --min-mass1 MIN_MASS1
                        Lower limit for mass of bigger BH.
  --max-mass1 MAX_MASS1
                        Upper limit for mass of bigger BH.
  --min-mass2 MIN_MASS2
                        Lower limit for mass of smaller BH.
  --max-mass2 MAX_MASS2
                        Upper limit for mass of smaller BH.
  --min-q MIN_Q         Lower limit for mass of smaller BH.
  --max-q MAX_Q         Upper limit for mass of smaller BH.
  --grid-spacing-mass1 GRID_SPACING_MASS1
                        Step size for sampling mass of bigger BH.
  --grid-spacing-mass2 GRID_SPACING_MASS2
                        Step size for samp

In [8]:
!gwnrtools_enigma_plan_calib_grid_and_make_dag --verbose \
    --min-mass1=3 \
    --max-mass1=100 \
    --min-mass2=3 \
    --max-mass2=100 \
    --choices-f-lower=15,30 \
    --run-dir=/home/prayush/TESTME

INFO:root:Going to setup run with:
INFO:root:Mass1 $\in$ [3.0, 100.0]Msun
INFO:root:Mass2 $\in$ [3.0, 100.0]Msun
INFO:root:Mass-ratio $\in$ [1.0, 10.0]Msun
INFO:root:... with spacing: 1.0, 1.0
INFO:root:Will sample lower freq cutoff from: [15. 30.]
INFO:root:Will sample PSD from: ['aLIGOZeroDetHighPower']
INFO:root:Will setup 100 calculations per job
INFO:root: Setting up the workflow in /home/prayush/TESTME now ...

INFO:root:.. directories made.
INFO:root: .. parameter grid constructed.
INFO:root: .. condor submission file written.
INFO:root: .. DAG written.
INFO:root:All Done in 1.4547419548 seconds


## `gwnrtools_enigma_sample_parameters`

**Code plan**

* `gwnrtools_enigma_sample_parameters`:
 - Inputs:
   - Parameter file
 - Meat:
   - Iterate over {mass1, mass2, f_lower, psd_choices}
   - MCMC over $\{\omega_\mathrm{attach}, \mathrm{PNO}\}$
 - Output:
   - Text file with columns:
     - [1] mass-ratio
     - [2] total-mass
     - [3] mass1
     - [4] mass2
     - [5] ecc value
     - [6] PN order
     - [7] omega_attach
     - [8] overlap
     - [9] Re[unphase-maximized overlap]
     - [10] Im[unphase-maximized overlap]
     - [11] norm1
     - [12] norm2

In [12]:
os.chdir(run_directory)

In [20]:
# Fake inputs
param_file = 'input/parameters_006000.hdf'
sample_rate = 4096
time_length = 32
f_lower     = 15.

In [14]:
delta_t = 1. / sample_rate
delta_f = 1. / time_length
N = sample_rate * time_length
n = N / 2 + 1

In [15]:
fin = h5py.File(param_file, 'r')

'/media/prayush/Data/research/ROM_data/'

* Capture inputs

In [16]:
inputs = {}
with h5py.File(param_file, 'r') as fin:
    for k in fin:
        inputs[k] = fin[k][()]

* Function for the log(probability) calculation to be used by the MCMC sampler

In [17]:
def get_sampler(log_probability, myarglist,
                nwalkers = 32,
                omega_attach_lims = [0.01, 0.1],
                PNO_choices = [6, 7, 8, 9, 10, 11, 12]):
    """
Function to Initialize and burn-in MCMC sampler
    """
    # Setup hyper-parameters for the sampler
    ndim = 2
    
    # Initialize emsemble sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=myarglist)
    
    # Run the sampler for a few steps to burn-in,
    # ie erase memory of the starting locations
    p0 = np.hstack([np.random.uniform(omega_attach_lims[0],
                                      omega_attach_lims[-1],
                                      (nwalkers,1)),
                    np.random.uniform(PNO_choices[0] - 0.5,
                                      PNO_choices[-1] + 0.5,
                                      (nwalkers,1))])    
    state = sampler.run_mcmc(p0, 100)
    sampler.reset()
    return sampler, state

def write_output_from_sampler(output_file_name, sampler, myargs):
    """
Function to write output of ONE MCMC sampler to txt file
    """
    m1, m2, f_low, _ = myargs
    with open(output_file_name, 'w+') as fout:
        for p in sampler.chain():
            # extracted sampled params and likelihood value
            pno = 1
            omega_att = -1
            mch = 1e9
            # write samples
            fout.write(\
                "{0:.18e}\t{1:.18e}\t{2:.18e}\t{3:.18e}\t{4:.18e}\t{5:.18e}\t{6:.18e}\t{7:.18e}".format(\
                    m1 / m2, m1 + m2, m1, m2, 0.0, pno, omega_att, mch))
    return

def log_prior_enigma(theta,
                     omega_attach_lims = [0.01, 0.1],
                     PNO_choices = [6, 7, 8, 9, 10, 11, 12]):
    '''
Priors:
-------

omega_attach : [0.01, 0.1]
PNO : [6, 7, 8, 9, 10, 11, 12]
    '''
    omega_attach, PNO = theta
    PNO = int(np.round(PNO))
    if PNO not in PNO_choices:
        return -np.inf
    if omega_attach < omega_attach_lims[0] or omega_attach > omega_attach_lims[-1]:
        return -np.inf
    
    return 0.0

def log_likelihood(theta, mass1, mass2, f_lower, sample_rate, psd):
    # extract MCMC parameters
    omega_attach, PNO = theta
    PNO = int(np.round(PNO))
    
    # Use BASH MAGIC TO PASS MCMC parameters TO ENIGMA
    os.environ['OMEGA_ATTACH'] = '{0:.12f}'.format(omega_attach)
    os.environ['PN_ORDER']     = '{0:d}'.format(PNO)
    
    dt = 1. / sample_rate
    df = psd.delta_f
    N = int(sample_rate / psd.delta_f)
    
    # Generate ENIGMA wave
    try:
        h1 = get_td_waveform(approximant='ENIGMA',
                         mass1=mass1,
                         mass2=mass2,
                         f_lower=f_lower,
                         delta_t=dt)
    except:
        logging.error("Could not generate ENIGMA wave..")
    h1 = make_padded_frequency_series(h1, N, df)
    
    # Generate EOB wave
    try:
        h2 = get_fd_waveform(approximant='SEOBNRv4_ROM',
                         mass1=mass1,
                         mass2=mass2,
                         f_lower=f_lower,
                         delta_f=df)
    except:
        logging.error("Could not generate EOB wave..")
    h2 = make_padded_frequency_series(h2, N, df)
    
    # Undo BASH MAGIC TO PASS MCMC parameters TO ENIGMA
    os.environ['OMEGA_ATTACH'] = ''
    os.environ['PN_ORDER']     = ''
    
    # Compute inner prodcut
    log_like, _ = match(h1, h2, psd=psd, low_frequency_cutoff=f_lower)
    return log_like

def log_prob(theta, mass1, mass2, f_lower, sample_rate, psd):
    log_prior = log_prior_enigma(theta)
    if log_prior > -np.inf:
        print("YAY: sampling within prior...")
        return log_likelihood(theta, mass1, mass2, f_lower, sample_rate, psd) + log_prior
    return log_prior

In [21]:
psd = from_string(inputs['psd'][0], n, delta_f, f_lower)
psd = make_padded_frequency_series(psd, N, delta_f) # from global settings

In [ ]:
log_likelihood((0.06, 10),
              20., 18., 15., 4096, psd)

* Main workflow

In [ ]:
num_steps = 5

samplers = {}

# Loop over all unique MCMC jobs
for idx in range(len(inputs[inputs.keys()[0]])):
    if True:
        mass1 = inputs['mass1'][idx]
        mass2 = inputs['mass2'][idx]
        f_lower = inputs['f_lower'][idx]    
        
        psd = from_string(inputs['psd'][idx], n, delta_f, f_lower)
        psd = make_padded_frequency_series(psd, N, delta_f) # from global settings
        
        # RUN THE SAMPLER
        logging.warn("Initializing the MCMC sampler and burning-in")
        samplers[idx] = get_sampler([mass1, mass2, f_lower, sample_rate, psd])
        
        logging.warn("Running the MCMC sampler for {0} steps".format(num_steps))
        s, state = samplers[idx]
        s.run_mcmc(state, num_steps)
        
        # WRite output from the sampler
        write_output_from_sampler("OUT_FILE_NAME", s, [mass1, mass2, f_lower, psd])
        
        # Early stop
        if idx >= 0:
            break
    else:
        logging.warn("Error in parameter set[{0}]: m1={1:.3f}, m2={2:.3f}, f_low={3:.0f}".format(idx,
                                                                                                mass1,
                                                                                                mass2,
                                                                                                f_lower))

In [ ]:
import lalsimulation

In [ ]:
lalsimulation.SEOBNRv1

In [ ]:
lalsimulation.ENIGMA

In [ ]:
state?

In [ ]:
s, state = samplers[0]

In [ ]:
state = s.run_mcmc(p0, 100)

In [ ]:
np.shape(state[0])

In [ ]:
state[0]

In [ ]:
s.run_mcmc(state[0], 10000)

In [ ]:
s.run_mcmc?

In [ ]:
log_prior_enigma([0.09, 12.51])

In [ ]:
0 > -np.inf

### Visualize the posterior

In [ ]:
import matplotlib.pyplot as plt
%pylab inline

samples = sampler.get_chain(flat=True)
plt.axvline(0)
plt.hist(samples[:, 0], 100, color="k", histtype="step")
plt.xlabel(r"$\theta_1$")
plt.ylabel(r"$p(\theta_1)$")
plt.gca().set_yticks([]);

### Check diagnostics

In [ ]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))

In [ ]:
print(
    "Mean autocorrelation time: {0:.3f} steps".format(
        np.mean(sampler.get_autocorr_time())
    )
)